Tóm tắt thông tin mô hình:
- Mô hình sử dụng GXBoost
- Kiểm định whole dataset cross-validation: Để tận dụng toàn bộ tập dataset để train mô hình (vì dataset hiện chỉ giới hạn  4k dòng)


## TD forecast on python

In [109]:
# %load_ext autoreload
# %autoreload 2
# import pyodbc
# import pandas as pd
# import numpy as np

# import xgboost as xgb
# # from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
# # from sklearn.metrics import r2_score, mean_squared_error
# # from sklearn.linear_model import LinearRegression
# # from sklearn import model_selection

# import pickle

# server = '10.16.157.42'
# database = 'RB_DATA'

# connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes'
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()
# sql_query = '''
#     SELECT *
#     FROM USER_DATA.HIENNPD3.FCST_EOP_TD_MAINMODEL 
#     WHERE PERIOD LIKE '2023-%' OR PERIOD LIKE  '2024-%'
#     ORDER BY PERIOD DESC
# '''
# cursor.execute(sql_query)  
# rows = cursor.fetchall()

# columns = [column[0] for column in cursor.description]
# df = pd.DataFrame.from_records(rows, columns=columns)

# cursor.close()
# conn.close() 


In [110]:
import pandas as pd
import numpy as np

import xgboost as xgb
# from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
# from sklearn.metrics import r2_score, mean_squared_error
# from sklearn.linear_model import LinearRegression
# from sklearn import model_selection

import pickle

df = pd.read_csv(r'C:\Users\admin\Desktop\Github\Crawling\Dummy.csv')
df

,PERIOD,TERM_,CURRENCY_2,NET,EOP_CM,EOP_LM,UMASS_MIN,MAF_MIN,AF_MIN,UMASS_MAX,...,RANKING_UMASS_VPB,RANKING_MAF_VPB,RANKING_AF_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,VOF_VOL,CIE_VOL,ADB_VOL
0,2024-03,From 18M,VND,NaN,NaN,NaN,2.99,2.99,2.99,4.7,...,2.0,3.0,3.0,4.51,4.07,3.0,4.5,NaN,NaN,NaN
1,2024-03,1-3M,VND,NaN,NaN,NaN,1.73,1.73,1.73,2.9,...,5.0,6.0,6.0,1.88,2.65,3.0,4.5,NaN,NaN,NaN
2,2024-03,6-9M,VND,NaN,NaN,NaN,2.99,2.99,2.99,4.7,...,2.0,3.0,3.0,4.51,4.07,3.0,4.5,NaN,NaN,NaN
3,2024-03,less than 1M,VND,NaN,NaN,NaN,1.73,1.73,1.73,5.1,...,5.0,6.0,6.0,1.47,1.75,3.0,4.5,NaN,NaN,NaN
4,2024-03,4-5M,VND,NaN,NaN,NaN,1.73,1.73,1.73,2.9,...,5.0,6.0,7.0,1.88,2.65,3.0,4.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2018-11,6-9M,VND,5494.340297,34347.522130,28853.181830,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
803,2018-11,From 18M,VND,9.166816,3892.125851,3882.959036,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,2018-11,4-5M,VND,20.093404,205.921888,185.828484,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,2018-11,less than 1M,VND,-14748.943210,288.208192,15037.151400,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df['PERIOD'] = pd.to_datetime(df['PERIOD']) 
df_v2 = df 
df_v2 = df_v2[['PERIOD', 'CURRENCY_2', 'TERM_'
               , 'EOP_CM', 'EOP_LM'
               ,'UMASS_MIN','UMASS_MAX','UMASS_VPB','RANKING_UMASS_VPB'
               , 'INTERBANK_SHORT', 'INTERBANK_LONG', 'DISCOUNTRATE'
               , 'REFINANCINGRATE', 'CIE_VOL', 'ADB_VOL']]
df_v2['YEAR'] = df_v2['PERIOD'].dt.year 
df_v2['MONTH'] = df_v2['PERIOD'].dt.month 
df_v2['NO_ACT_DATE'] = df_v2['PERIOD'].dt.daysinmonth

C:\Users\admin\AppData\Local\Temp\ipykernel_2424\3472777732.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v2['YEAR'] = df_v2['PERIOD'].dt.year
C:\Users\admin\AppData\Local\Temp\ipykernel_2424\3472777732.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v2['MONTH'] = df_v2['PERIOD'].dt.month
C:\Users\admin\AppData\Local\Temp\ipykernel_2424\3472777732.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [112]:
from datetime import datetime, timedelta

def get_eop_lm(year, month, term, currency, dataset_name):
    previous_month = datetime(year, month, 1) - timedelta(days=1)
    previous_year = previous_month.year
    previous_month = previous_month.month

    eop_lm = dataset_name.loc[(dataset_name['YEAR'] == previous_year)
                              & (dataset_name['MONTH'] == previous_month)
                              & (dataset_name['TERM_'] == term)
                              & (dataset_name['CURRENCY_2'] == currency), 'EOP_CM'].values

    if len(eop_lm) > 0:
        return eop_lm[0]
    else:
        return np.nan

### 1. Forecast EOP_CM -> Term

In [113]:
df_v3 = df_v2.groupby(['YEAR', 'MONTH', 'NO_ACT_DATE', 'CURRENCY_2', 'TERM_']).agg({'EOP_CM': 'sum'
                                                            , 'EOP_LM': 'sum'                                                                         
                                                            , 'UMASS_MIN': 'max'
                                                            , 'UMASS_MAX': 'max'
                                                            , 'UMASS_VPB': 'max'
                                                            , 'RANKING_UMASS_VPB': 'max'

                                                            , 'INTERBANK_SHORT': 'max'
                                                            , 'INTERBANK_LONG': 'max'
                                                            , 'DISCOUNTRATE': 'max'
                                                            , 'REFINANCINGRATE': 'max'
                                                            
                                                            , 'CIE_VOL': 'sum'
                                                            , 'ADB_VOL': 'sum'}).reset_index()
float_columns = ['EOP_CM','EOP_LM', 'UMASS_MIN', 'UMASS_MAX', 'UMASS_VPB', 'RANKING_UMASS_VPB'
                 , 'INTERBANK_SHORT' , 'INTERBANK_LONG', 'DISCOUNTRATE', 'REFINANCINGRATE' , 'CIE_VOL']

df_v3[float_columns] = df_v3[float_columns].astype(float)


In [114]:
df_v3['EOP_LM_FUNC'] = df_v3.apply(lambda row: row['EOP_LM'] if ((row['YEAR'] == 2023) and (row['MONTH'] == 1)) else get_eop_lm(row['YEAR'],
                                                                                                                                row['MONTH'],
                                                                                                                                row['TERM_'],
                                                                                                                                row['CURRENCY_2'], df_v3),
                                                                                                                                axis=1)
df_v3['CIE_Rate'] = np.where( (df_v3['CIE_VOL'] == 0) | (df_v3['CIE_VOL'] is None) | (df_v3['ADB_VOL'] == 0) |  (df_v3['ADB_VOL'] is None) , 
                             0,
                             (df_v3['CIE_VOL'] / (-df_v3['ADB_VOL'])) * (365/ df_v3['NO_ACT_DATE'] ) * 100 
                             )

In [115]:
mapping_TERM_ = {'less than 1M': 0,
                '1-3M': 1,
                '4-5M': 2,
                '6-9M': 3,
                '10-11M': 4,
                '12-18M': 5,
                'From 18M': 6
                }
mapping_CURRENCY_2 = {'VND': 0, 'NON-VND': 1}

In [116]:
categorical_cols = [ 'CURRENCY_2', 'TERM_']

def encode_engine(df_name, column_name):
    if column_name == 'CURRENCY_2':
        map_rule = mapping_CURRENCY_2
    elif column_name == 'TERM_':
        map_rule = mapping_TERM_
    df_name[column_name] = df_name[column_name].map(map_rule)
    return df_name

for x in categorical_cols:
    df_v4 = encode_engine(df_v3, x)

In [117]:
from sklearn.model_selection import GridSearchCV

df_train = df_v4[~(df_v4['YEAR'] == 2024)]
x_train = df_train.drop(['EOP_CM'], axis=1)
y_train = df_train['EOP_CM']

df_test = df_v4[(df_v4['YEAR'] == 2024) & df_v4['MONTH'] != 3]
x_test = df_test.drop(['EOP_CM'], axis=1)
y_test = df_test['EOP_CM']

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0]
}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=3)
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_

xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(x_train, y_train)

best_model = grid_search.best_estimator_
# Lưu mô hình vào tệp
filename = 'best_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)

y_train_pred = xgb_model.predict(x_train)
y_test_pre =  xgb_model.predict(x_test)
df_train['FCST_EOP_CM'] = y_train_pred
df_test['FCST_EOP_CM'] = y_test_pre
score_train = xgb_model.score(x_train, y_train)
score_test = xgb_model.score(x_test, y_test)

score_test, score_train


C:\Users\admin\AppData\Local\Temp\ipykernel_2424\529840346.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['FCST_EOP_CM'] = y_train_pred


(0.9835482192473756, 0.9994894785813901)

In [118]:
x_train

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
0,2018,11,30,1,1,490.938400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
1,2018,11,30,1,5,1689.114219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
2,2018,11,30,1,2,12.113397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
3,2018,11,30,1,3,356.655184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
4,2018,11,30,1,6,168.746705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023,12,31,0,5,63522.272820,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-489.652166,66091.207770,67182.521860,8.723187
763,2023,12,31,0,2,387.146713,2.6,4.35,3.6,5.0,1.33,4.54,3.0,4.5,-1.638594,478.706360,469.056092,4.030262
764,2023,12,31,0,3,114831.296000,3.9,5.50,4.8,7.0,5.08,6.68,3.0,4.5,-718.561196,131654.763000,127401.309600,6.426261
765,2023,12,31,0,6,1898.013270,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-14.027270,1908.714485,1925.689075,8.652933


In [119]:
x_test

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
0,2018,11,30,1,1,490.938400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
1,2018,11,30,1,5,1689.114219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
2,2018,11,30,1,2,12.113397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
3,2018,11,30,1,3,356.655184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
4,2018,11,30,1,6,168.746705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2024,3,31,0,5,0.000000,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,54153.532690,0.0
803,2024,3,31,0,2,0.000000,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,435.287744,0.0
804,2024,3,31,0,3,0.000000,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,153359.218200,0.0
805,2024,3,31,0,6,0.000000,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,1930.516589,0.0


In [125]:
x_test.to_csv(r'C:\Users\admin\Desktop\Github\Crawling\x_test.csv', index=False)

In [121]:
best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [122]:
x_va = x_test[(x_test['MONTH']==3)& (x_test['YEAR']==2024)& (x_test['CURRENCY_2']== 1)]
x_va

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
793,2024,3,31,1,1,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,311.190877,0.0
794,2024,3,31,1,4,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,NaN,0.0
795,2024,3,31,1,5,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,212.802887,0.0
796,2024,3,31,1,2,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,2.939174,0.0
797,2024,3,31,1,3,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,179.575271,0.0
798,2024,3,31,1,6,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,48.022415,0.0
799,2024,3,31,1,0,0.0,1.73,5.1,2.5,5.0,1.47,1.75,3.0,4.5,0.0,0.0,13.178794,0.0


#### Giả định input CIE được truyền vào
Def function để thực hiện predict trong quá trình người nhập dữ liệu

In [123]:
INPUT_CIE_U1M = 0.1
INPUT_CIE_1M3M = 3.1
INPUT_CIE_4M5M = 3.2
INPUT_CIE_6M9M = 3.5
INPUT_CIE_10M11M = 4.5
INPUT_CIE_12M18M = 6.0
INPUT_CIE_OV18M = 7.0

mapping_input = { 0 : INPUT_CIE_U1M,
                  1 : INPUT_CIE_1M3M,
                  2 : INPUT_CIE_4M5M,
                  3 : INPUT_CIE_6M9M,
                  4 : INPUT_CIE_10M11M,
                  5 : INPUT_CIE_12M18M,
                  6 : INPUT_CIE_OV18M
                }

x_va['CIE_Rate'] =  x_va['TERM_'].map(mapping_input)
x_va

C:\Users\admin\AppData\Local\Temp\ipykernel_2424\412722065.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_va['CIE_Rate'] =  x_va['TERM_'].map(mapping_input)


,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
793,2024,3,31,1,1,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,311.190877,3.1
794,2024,3,31,1,4,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,NaN,4.5
795,2024,3,31,1,5,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,212.802887,6.0
796,2024,3,31,1,2,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,2.939174,3.2
797,2024,3,31,1,3,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,179.575271,3.5
798,2024,3,31,1,6,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,48.022415,7.0
799,2024,3,31,1,0,0.0,1.73,5.1,2.5,5.0,1.47,1.75,3.0,4.5,0.0,0.0,13.178794,0.1


In [124]:
x_va['EOP_CB_FCST'] =  xgb_model.predict(x_va)

x_va

C:\Users\admin\AppData\Local\Temp\ipykernel_2424\1763910430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_va['EOP_CB_FCST'] =  xgb_model.predict(x_va)


,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate,EOP_CB_FCST
793,2024,3,31,1,1,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,311.190877,3.1,85.850746
794,2024,3,31,1,4,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,NaN,4.5,407.372437
795,2024,3,31,1,5,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,212.802887,6.0,76.889084
796,2024,3,31,1,2,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,2.939174,3.2,99.133484
797,2024,3,31,1,3,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,179.575271,3.5,134.966309
798,2024,3,31,1,6,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,48.022415,7.0,134.966309
799,2024,3,31,1,0,0.0,1.73,5.1,2.5,5.0,1.47,1.75,3.0,4.5,0.0,0.0,13.178794,0.1,-174.994446


### 2. Forecast NET -> movement_status

### 3. Combine các giá trị ở các năm vào một bảng
- Combine
- Xuất giá trị vào csv
